In [ ]:
!pip3 install pandas

In [1]:
!wget https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-01-TTD_target_download.txt -O P1-01-TTD_target_download.txt
!wget https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-02-TTD_drug_download.txt -O P1-02-TTD_drug_download.txt

--2024-04-16 12:13:31--  https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-01-TTD_target_download.txt
Resolving db.idrblab.net (db.idrblab.net)... 47.88.56.212
Connecting to db.idrblab.net (db.idrblab.net)|47.88.56.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8672051 (8.3M) [text/plain]
Saving to: ‘P1-01-TTD_target_download.txt’

P1-01-TTD_target_do 100%[===================>]   8.27M  6.35MB/s    in 1.3s    

2024-04-16 12:13:33 (6.35 MB/s) - ‘P1-01-TTD_target_download.txt’ saved [8672051/8672051]

--2024-04-16 12:13:33--  https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-02-TTD_drug_download.txt
Resolving db.idrblab.net (db.idrblab.net)... 47.88.56.212
Connecting to db.idrblab.net (db.idrblab.net)|47.88.56.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12463915 (12M) [text/plain]
Saving to: ‘P1-02-TTD_drug_download.txt’

P1-02-TTD_drug_down 100%[===================>]  11.89M  6.53MB/s    in 1.8s 

In [10]:
import pandas as pd

def parse_section(section):
    """
    Parse a section of the file and extract the data.
    """
    base_data = {
        'ttd_target_id': ''
    }

    lines = section.split('\n')
    for line in lines:
        parts = line.split('\t')
        if parts[1] == 'TARGETID':
            base_data['ttd_target_id'] = parts[2]
    
    druginfo_data = []
    for line in lines:
        parts = line.split('\t')
        if parts[1] == 'DRUGINFO':
            drug_data = base_data.copy()
            drug_data.update({
                'source_id': parts[2],
                'source_name': parts[3],
                'source_type': 'Compound',
                'target_type': 'Gene',
                'relation_type': 'DRUGBANK::target::Compound:Gene',
                'ttd_source_id': parts[2],
                'resource': 'TTD'
            })
            druginfo_data.append(drug_data)
    return druginfo_data

def process_file(filepath):
    """
    Process a file and extract all the data.
    """
    with open(filepath, 'r') as file:
        for i in range(0, 32):
            next(file)

        content = file.read().strip()
        sections = content.split("\n\n")

    all_data = []
    for section in sections:
        section_data = parse_section(section)
        all_data.extend(section_data)

    df = pd.DataFrame(all_data)

    df.to_csv("P1-01-TTD_target_download_extracted.csv", index=False)

process_file('P1-01-TTD_target_download.txt')

In [16]:
import pandas as pd

df1 = pd.read_csv("P1-01-TTD_target_download_extracted.csv")
df2 = pd.read_csv("../idmapping/ttd_drug_id.tsv", sep="\t")
df3 = pd.read_csv("../idmapping/ttd_gene_id.tsv", sep="\t")

merged_df = pd.merge(df1, df2, left_on="source_id", right_on="TTDDRUID", how="left")
merged_df["source_id"] = merged_df["Drug_id"]
merged_df = merged_df.drop(columns=["TTDDRUID", "Drug_id"])

merged_df = pd.merge(
    merged_df,
    df3[["ttd_target_id", "target_id", "target_name"]],
    left_on="ttd_target_id",
    right_on="ttd_target_id",
    how="left",
)

# Remove all rows if the target_id or source_id is missing
print("Number of rows before filtering: ", len(merged_df), merged_df.columns)
invalid_rows = merged_df[
    (merged_df["target_id"].isna() | merged_df["source_id"].isna())
]
merged_df = merged_df[merged_df["target_id"].notna() & merged_df["source_id"].notna()]
print("Number of rows after filtering: ", len(merged_df))

invalid_rows.to_csv("invalid_ttd_drug_target.tsv", index=False, sep="\t")
merged_df.to_csv("formatted_ttd_drug_target.tsv", index=False, sep="\t")

print(
    "Merge and replacement completed. The result has been saved as 'formatted_ttd_drug_target.tsv'."
)

Number of rows before filtering:  46867 Index(['ttd_target_id', 'source_id', 'source_name', 'source_type',
       'target_type', 'relation_type', 'ttd_source_id', 'resource',
       'target_id', 'target_name'],
      dtype='object')
Number of rows after filtering:  35176
Merge and replacement completed. The result has been saved as 'formatted_ttd_drug_target.tsv'.
